*The section on "Data retrieval using chembl_downloader" has been updated on 26th Jan 2024 - many thanks for the comment from cthoyt*.

##### **Brief introduction**

This post was really just an addition towards the last random forest (RF) post. It was mainly inspired by this paper [@esposito2021] from [rinikerlab](https://github.com/rinikerlab)[^1]. It was nice to complete the RF series by adding a RF classifier since last post was only on a regressor. Another thing was that imbalanced datasets were common in drug discovery projects, learning different strategies to deal with them was also very useful. While working on this post, I also came across a few other packages that I haven't used before so I've included them all down below.

[^1]: h/t: Greg Landrum for his comment on Mastodon for the last RF post (which led to this follow-up post)

<br>

##### **Overview of post**

-   Data sourcing via *chembl_downloader*
-   Minor data preprocessing using own little script and also SMILES checker from *scikit_mol*
- *scikit-learn*'s RandomForestClassifier()
-   Dealing with imbalanced dataset in RF classifiers by using *ghostml*
-   A small section on plotting receiver operating characteristic (ROC) curves

<br>

##### **Importing libraries**


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chembl_downloader
from chembl_downloader import latest, queries, query
from rdkit.Chem import Descriptors
import datamol as dm
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay, roc_curve
from scikit_mol.utilities import CheckSmilesSanitazion
import ghostml

<br>

##### **Data retrieval using chembl_downloader**

chembl_downloader was something I wanted to try a while back. I've tried manual download and chembl_webresource_client previously, and they were probably not the best strategies for data reproducibility. The idea of chembl_downloader was to generate a reproducible ChEMBL data source. It involved some SQL at the beginning to specify the exact type of data needed, so some SQL knowledge were required. Other uses for this package were elaborated much more clearly in its GitHub repository at [https://github.com/cthoyt/chembl-downloader](https://github.com/cthoyt/chembl-downloader). One of the reference notebooks that I've used could be reached [here](https://github.com/cthoyt/chembl-downloader/blob/main/notebooks/drug-indications.ipynb) (more available in its repository).

What I did was shown below.


In [ ]:
# Show the latest version of ChEMBL used
latest_version = latest()
print(f"The latest ChEMBL version is: {latest_version}")

In [ ]:
# Query chembl_downloader to show SQL required 
# to extract ChEMBL data for a specific protein target
# e.g. target_chembl_id for acetylcholinesterase (AChE): CHEMBL220
queries.markdown(queries.get_target_sql(target_id="CHEMBL220", target_type="SINGLE PROTEIN"))

The code below took several minutes to run.


In [ ]:
# Following data were 4.41GB in total from the SQL query
# Added an extra line - MOLECULE_DICTIONARY.max_phase 
# into sql to show max phases of compounds targeting AChE

sql = """
SELECT
    ASSAYS.chembl_id              AS assay_chembl_id,
    TARGET_DICTIONARY.target_type,
    TARGET_DICTIONARY.tax_id,
    TARGET_DICTIONARY.chembl_id,
    COMPOUND_STRUCTURES.canonical_smiles,
    MOLECULE_DICTIONARY.chembl_id AS molecule_chembl_id,
    MOLECULE_DICTIONARY.max_phase,
    ACTIVITIES.standard_type,
    ACTIVITIES.pchembl_value
FROM TARGET_DICTIONARY
     JOIN ASSAYS ON TARGET_DICTIONARY.tid == ASSAYS.tid
     JOIN ACTIVITIES ON ASSAYS.assay_id == ACTIVITIES.assay_id
     JOIN MOLECULE_DICTIONARY ON MOLECULE_DICTIONARY.molregno == ACTIVITIES.molregno
     JOIN COMPOUND_STRUCTURES ON MOLECULE_DICTIONARY.molregno == COMPOUND_STRUCTURES.molregno
WHERE TARGET_DICTIONARY.chembl_id = 'CHEMBL220'
    AND ACTIVITIES.pchembl_value IS NOT NULL
    AND TARGET_DICTIONARY.target_type = 'SINGLE PROTEIN'
"""

#df = chembl_downloader.query(sql)

I've updated how I retrieved and saved the ChEMBL data - code as suggested by cthoyt. This would be a better and more reproducible way for anyone who might be interested in re-running this notebook.


In [ ]:
# Suggestions
from pathlib import Path

# Pick any directory, but make sure it's relative to your home directory
directory = Path.home().joinpath(".data", "blog")
# Create the directory if it doesn't exist
directory.mkdir(exist_ok=True, parents=True)

# Create a file path that corresponds to the version, since this could change
path = directory.joinpath(f"chembl_d_ache_{latest_version}.tsv")

if path.is_file():
    # If the file already exists, load it
    df_ache = pd.read_csv(path, sep=',')
else:
    # If the file doesn't already exist, make the query then cache it
    df_ache = chembl_downloader.query(sql)
    df_ache.to_csv(path, sep=",", index=False)

<br>

##### **Some data cleaning**

Minor cleaning and preprocessing were done for this post only, as the focus was more on dealing with imbalanced dataset in RF classifier. Since I used a different way to retrieve ChEMBL data this time, the dataset used here might be slightly different from the one used in previous post.

<br>

###### **mol_prep.py**

I've more or less accumulated small pieces of code over time, and I've decided to compile them into a Python script. The idea was to remove most function code in the post to avoid repeating them all the time since they've been used frequently in the last few posts. The script would be saved into the [RF repository](https://github.com/jhylin/ML2-2_random_forest), and would still be considered as a "work-in-progress" script (needs more work in the future).


In [ ]:
## Trial own mol_prep.py script
from mol_prep import preprocess, rdkit_2d_descriptors

In [ ]:
## Preprocess/standardise molecules
# Running preprocess function 
df_ache = df_ache.copy()
df_prep = df_ache.apply(preprocess, axis = 1)
df_prep.head(3)

<br>

###### **scikit_mol**

scikit_mol was a package originated from RDKit UGM hackathon in 2022. This [blog post](https://www.cheminformania.com/scikit-mol-easy-embedding-of-rdkit-into-scikit-learn/) elaborated further on its functions and uses in machine learning. For this post I've only used it for a very small portion, mainly to check for missing SMILES or errors in SMILES (kind of like double checking whether the preprocess function code worked as expected). It could be integrated with *scikit-learn*'s pipeline method for multiple estimators. Its GitHub Repository link: [https://github.com/EBjerrum/scikit-mol](https://github.com/EBjerrum/scikit-mol) - I've referred to this [reference notebook](https://github.com/EBjerrum/scikit-mol/blob/main/notebooks/05_smiles_sanitaztion.ipynb) while working on this post.


In [ ]:
# Quick simple way to check for missing SMILES
print(f'Dataset contains {df_prep.standard_smiles.isna().sum()} unparsable mols')

In [ ]:
# Checking for invalid SMILES using scikit_mol
smileschecker = CheckSmilesSanitazion()
smileschecker.sanitize(list(df_prep.standard_smiles))

# Showing SMILES errors
smileschecker.errors

It showed no errors in SMILES (errors should be listed in the code cell output).


In [ ]:
## Generate RDKit 2D descriptors/fingerprints
# Running rdkit_2d_descriptors function
df_2d = rdkit_2d_descriptors(df_prep)
df_2d.head(3)

In [ ]:
# Merge dataframes df_prep & df_2d via index
df_merge = pd.merge(
    df_prep[["max_phase", "molecule_chembl_id"]],
    df_2d,
    left_index=True,
    right_index=True
)
print(df_merge.shape)
df_merge.head(3)

A different spreads of max phases were shown this time in the dataframe, as the SQL query mainly used IC50, whereas last post was strictly limited to Ki via ChEMBL web resource client. Other likely reason was that in the decision tree series, I've attempted data preprocessing at a larger scale so some data were eliminated. It appeared that there were more max phase 4 compounds here than last time (note: null compounds were not shown in the value counts as it was labelled as "NaN", it should be the largest max phase portion in the data).


In [ ]:
# Find out counts of each max phase
df_merge.value_counts("max_phase")

I then tried searching for the chembl_id of the 10 max phase 4 compounds used in the last post in the dataframe (df_merge).


In [ ]:
# Previously used 10 max phase 4 compounds
# donepezil = CHEMBL502 & galantamine = CHEMBL659
list_mp4 = ["CHEMBL95", "CHEMBL1128", "CHEMBL640", "CHEMBL502", "CHEMBL481", "CHEMBL360055", "CHEMBL1025", "CHEMBL659", "CHEMBL1200970", "CHEMBL1677"]

# Search for compounds in list_mp4 within df_merge's "molecule_chembl_id" column
# using Series.isin
df_prev = df_merge.loc[df_merge["molecule_chembl_id"].isin(list_mp4)]
df_prev.head()

There were many duplicates of compounds.


In [ ]:
print(df_prev.shape)
df_prev.value_counts("molecule_chembl_id")

In [ ]:
# Dropping duplicated compound via chembl IDs in the main df
df_merge_new = df_merge.drop_duplicates(subset=["molecule_chembl_id"], keep="first")
print(df_merge_new.shape)
df_merge_new.head()

In [ ]:
# Making sure previously used 10 max phase 4 compounds could be found in df_merge_new
df_mp4 = df_merge_new.loc[df_merge_new["molecule_chembl_id"].isin(list_mp4)]
df_mp4

In [ ]:
# note: compounds with max phase 0 not shown in the count
df_merge_new.value_counts("max_phase")

<br>

##### **Model building**

The aim of this post was to model and classify the max phases of ChEMBL small molecules, i.e. whether the compounds in the testing set (consisted of max phase 0 or null compounds) might be eventually classified as max phase 4 or not. This was one of the approaches to answer the question in mind, and not the ultimate way to solve the problem (just thought to mention). The target was "max_phase" and features to be used were the various RDKit 2D descriptors (RDKit2D).

The steps I've taken to build the model were shown below:

1.  Re-labelled max phases as binary labels (e.g. max phase null as 0, max phase 4 as 1)


In [ ]:
# Re-label max phase NaN as 0
df_merge_new = df_merge_new.fillna(0)
df_merge_new.head()

2.  Splitted data into max phase null & max phase 4 (needing to re-label max phase 4 column only as 1, and not disrupting the labels of max phase 0 compounds)


In [ ]:
# Select all max phase null compounds
df_null = df_merge_new[df_merge_new["max_phase"] == 0]
print(df_null.shape)
df_null.head()

In [ ]:
# Using pd.DataFrame.assign to add a new column to re-label max_phase 4 as "1"
df_mp4_lb = df_mp4.assign(max_phase_lb = df_mp4["max_phase"] / 4)

# Using pd.DataFrame.pop() & insert() to shift added column to first column position
first_col = df_mp4_lb.pop("max_phase_lb")
df_mp4_lb.insert(0, "max_phase_lb", first_col)
df_mp4_lb.head()

In [ ]:
# Also create a new column max_phase_lb column for df_null 
# in order to merge 2 dfs later
df_null_lb = df_null.assign(max_phase_lb = df_null["max_phase"])
first_col_null = df_null_lb.pop("max_phase_lb")
df_null_lb.insert(0, "max_phase_lb", first_col_null)
df_null_lb.head()

In [ ]:
# Concatenate df_mp4_lb & df_null_lb
df_full = pd.concat([df_null_lb, df_mp4_lb])
df_full

3.  Defined X and y variables and trained RF classifier model

Eventually df_full contained 10 active compounds and 5256 inactive compounds (from the value counts).


In [ ]:
df_full.value_counts("max_phase_lb")

In [ ]:
# Defining X (features) & y (target)
X = df_full.iloc[:, 3:]
y = df_full.iloc[:, 0]

In [ ]:
# Checking right data were selected e.g. y as target
y

In [ ]:
# Convert both X & y to arrays
X = X.to_numpy()
y = y.to_numpy()

In [ ]:
# Using train_test_split() this time to split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

After data splitting, a RF classifier was trained with reference to this [notebook](https://github.com/rinikerlab/GHOST/blob/main/notebooks/example_GHOST.ipynb).


In [ ]:
# max_features = "sqrt" by default
rfc = RandomForestClassifier(max_depth=3, random_state=1, max_features="sqrt", oob_score=True)
rfc.fit(X_train, y_train)

4.  Extracted positive prediction probabilities for the testing set and showed confusion matrix with classification metrics


In [ ]:
test_probs = rfc.predict_proba(X_test)[:, 1]

Some reference links and explanations for area under the ROC curve and Cohen's Kappa.

**Area under the ROC curve**: [reference](https://scikit-learn.org/stable/modules/model_evaluation.html#receiver-operating-characteristic-roc) - the area under a curve plot between *sensitivity* or *recall* (percent of all 1s classified correctly by a classifier or true positive rate) and *specificity* (percent of all 0s classified correctly by a classifier, or equivalent to 1 - false positive rate or true negative rate) [@bruce2020]. It is useful for evaluating the performance of a classification model via comparing the true positive rate and false positive rate which are influenced by shifting the decision threshold. Area under the ROC is usually represented as a number ranging from 0 to 1 (1 being a perfect classifier, 0.5 or below meaning a poor, ineffective classifier)

**Cohen's Kappa score**: [reference](https://scikit-learn.org/stable/modules/model_evaluation.html#cohen-s-kappa) - a score that is used to measure the agreement of labelling between two annotators (usually between -1 and 1, the higher the score the better the agreement)

Rather than re-inventing the wheel, the following function code for calculating metrics of the RF model were adapted from [this notebook](https://github.com/rinikerlab/GHOST/blob/main/notebooks/example_GHOST.ipynb), from [GHOST repository](https://github.com/rinikerlab/GHOST). I have only added some comments for clarities, and also added a zero_division parameter for the classification_report to mute the warning message when the results ended up being 0 due to divisions by zero.


In [ ]:
def calc_metrics(y_test, test_probs, threshold = 0.5):
    # Target label assigned according to stated decision threshold (default = 0.5)
    # e.g. second annotator (expected label)
    scores = [1 if x>=threshold else 0 for x in test_probs]
    # Calculate area under the ROC curve based on prediction score
    auc = metrics.roc_auc_score(y_test, test_probs)
    # Calculate Cohen's Kappa score
    # e.g. y_test as first annotator (predicted label)
    kappa = metrics.cohen_kappa_score(y_test, scores)
    # Formulate the confusion matrix
    confusion = metrics.confusion_matrix(y_test, scores, labels = list(set(y_test)))
    print('thresh: %.2f, kappa: %.3f, AUC test-set: %.3f'%(threshold, kappa, auc))
    print(confusion)
    print(metrics.classification_report(y_test, scores, zero_division=0.0))
    return 

Note: roc_auc_score measures true positive and false positive rates, requiring binary labels (e.g. 0s and 1s) in the data

Then showed confusion matrix along with area under the ROC curve and Cohen's Kappa.


In [ ]:
calc_metrics(y_test, test_probs, threshold = 0.5)

To help with interpreting the confusion matrix, *scikit-learn* actually had a handy plotting code to visualise the matrix. 


In [ ]:
ConfusionMatrixDisplay.from_estimator(rfc, X_test, y_test)
plt.show()

It was very obvious that not all of the compounds were classified in the testing set. There were only 1052 compounds classified as true negative, and none in the testing set were labelled as true positive. The likely reason was due to the very imbalanced ratio of actives (only 10 max phase 4 which were labelled as "1" compounds) and inactives (5256 max phase 0 compounds). Besides the imbalanced dataset, the decision threshold was also normally set at 0.5, meaning the classifier was likely going to lose the chance to classify the true positive compounds due to the very skewed ratio of actives to inactives.

5.  Two approaches were used in the GHOST (generalized threshold shifting) paper:

    -   Approach 1 (out-of-bag method, more computer efficient, aimed for RF classifiers) based on [RDKit blog post](http://rdkit.blogspot.com/2018/11/working-with-unbalanced-data-part-i.html) or its [viewable notebook version](https://nbviewer.org/github/greglandrum/rdkit_blog/blob/master/notebooks/Working%20with%20unbalanced%20data%20part%201.ipynb) via nbviewer

    -   Approach 2 led to [GHOST](https://github.com/rinikerlab/GHOST) procedure with a goal to optimise and shift the decision threshold in any classification methods to catch the minor portion of actives (rather than the major portion of inactives) 

    - note: both approaches were shown to be performing similarly in the paper

I only used approach 2 here since the RDKit blog post had already explained approach 1 in depth. 

The next step involved extracting prediction probabilities from the RF classifier trained model.


In [ ]:
# Get the positive prediction probabilities of the training set
train_probs = rfc.predict_proba(X_train)[:, 1]

6.  Used GHOST strategy in a postprocessing way (note: last post used data re-sampling method in a preprocessing way)

The decision threshold were optimised by using *ghostml* code via testing various different thresholds, e.g. in spaces of 0.05 that ranged from 0.05 to 0.5. The most optimal threshold would have the most maximised Cohen's kappa.


In [ ]:
# Setting up different decision thresholds
thresholds = np.round(np.arange(0.05,0.55,0.05), 2)
thresholds

In [ ]:
# Looking for the best threshold with the most optimal Cohen's Kappa
new_threshold = ghostml.optimize_threshold_from_predictions(y_train, train_probs, thresholds, ThOpt_metrics = 'ROC') 

Using the calc_metrics function again on the newly-found or shifted decision threshold.


In [ ]:
calc_metrics(y_train, train_probs, threshold = new_threshold)

Here, after shifting the decision threshold with the most optimal Cohen's Kappa score, we could see an improved number of compounds labelled within the true negative class (increasing from 1052 to 4204), and more importantly, we could see the true positive class improved from 0 to 7 as well.

<br>

##### **Plotting ROC curves**

Time for some plots - I've shown two different ways to plot ROC curves below.

1.  Using *scikit-learn*

Testing set ROC curve - obviously, this was not a good classifier with a poor AUC.


In [ ]:
RocCurveDisplay.from_predictions(y_test, test_probs, plot_chance_level = True)

<br>

Training set ROC curve - this probably looked too good to be true or a textbook-standard ROC curve with AUC at 1.0.


In [ ]:
RocCurveDisplay.from_predictions(y_train, train_probs, plot_chance_level = True)

2.  Alternative method using matplotlib which reproduced a similar training set ROC plot:


In [ ]:
# Looking for true positive rate (tpr), false positive rate (fpr) & threshold
fpr, tpr, thresh = metrics.roc_curve(y_train, train_probs)
# Plotting
plt.figure()
# lw = linewidth
plt.plot(fpr, tpr, lw = 2)
# show random guessing line (threshold = 0.5)
plt.plot([0, 1], [0, 1], color = "g", lw = 2, linestyle="--")
plt.ylim([-0.05, 1.05])
plt.xlim([-0.05, 1.0])
plt.xlabel("specificity")
plt.ylabel("recall")
plt.show()

<br>

##### **Downsides and thoughts**

I wanted to mention that the testing set used here was most likely not the best ones to be used. There could be many overlaps or similarities between the training and testing sets, since they all came from ChEMBL database. For demonstration and learning purposes, I ended up using similar dataset as last time. Hopefully, I can try other open-source or public drug discovery datasets in the near future. 

The other thing to mention was that I should try different molecular fingerprints or descriptors as well, rather than only using RDKit2D, which might lead to different results. I should also probably slowly move onto using multiple datasets or targets in a project, which would likely make things more interesting. On the other hand, I also wanted to avoid this in order to make the topic of interest as clear and simple as possible for me or anyone who's trying to learn. 

<br>

##### **Acknowledgements**

I'd like to thank Riniker lab again for the GHOST paper, along with all the authors, contributors or developers for all of the software packages mentioned in this post, and also, huge thanks should also go to the authors of the reference notebooks mentioned in the post as well.